In [1]:
import os
os.chdir(r'D:\Work\text_reader\text-reader')

In [38]:
import re
# import datetime
import logging
import numpy as np

In [39]:
from datetime import datetime


In [26]:
import logging
from dateutil.parser import parse
from scripts.date_utils import extract_date

In [46]:
date = '03/04/23'

In [47]:
parsed_date = parse(date)

In [48]:
parsed_date.strftime("%Y-%m-%d")

'2023-03-04'

In [49]:
date_format = get_date_format([date])

In [50]:
date_format

'%d/%m/%y'

In [51]:
split_format = date_format.split('/')
year_index = split_format.index('%Y') if '%Y' in split_format else split_format.index('%y')
# 

In [52]:
year_index

2

In [56]:
valid_year = date.split('/')[year_index]

In [58]:
date_dt = str2datetime(date, date_format, valid_year, year_index)
date_dt

datetime.date(2023, 4, 3)

In [60]:
str(datetime2str(date_dt))

'2023-04-03'

In [59]:
def datetime2str(date):

    try:
        day = str(date.day).zfill(2)
        month = str(date.month).zfill(2)
        return '{}-{}-{}'.format(date.year, month, day)
    except Exception as err:
        logging.error('datetime to string failed: {}'.format(err))

    return ''

In [54]:
def str2datetime(date, date_format='%d/%m/%Y', valid_year=None, year_index=None):
    try:
        if valid_year and year_index:
            current_year = datetime.now().year
            year = date.split('/')[year_index]
            date = date.replace(year, valid_year) \
                if len(year) not in (2, 4) and current_year-10 > int(year) > current_year+1 \
                else date

        return datetime.strptime(date, date_format).date()

    except Exception as err:
        logging.error('Failed! String to datetime: %s' % err)
        return None

In [36]:
def get_date_format(dates):

    current_year = int(datetime.now().year)

    year_indices = [index for date in dates for index, value in enumerate(date.split('/')) if len(value) == 4
                    and current_year-10 <= int(value) <= current_year+2]
    date_format = '%d/%m/%Y' if any(year_indices) else '%d/%m/%y'

    year_index = max(np.unique(year_indices), key=year_indices.count) if year_indices else 2

    try:
        if year_index == 0:
            for date in dates:
                year, month, day = date.split('/')
                if 0 < int(month) <= 12 and 0 < int(day) <= 31:
                    date_format = '%Y/%m/%d'
                    break

        elif year_index == 2:
            for date in dates:
                day, month, year = date.split('/')
                # print('day: {}, month: {}, year:{}'.format(day, month, year))
                if int(month) > 12 >= int(day) >= 1:
                    date_format = '%m/%d/%Y' if len(year) == 4 else '%m/%d/%y'
                    break

    except Exception as error:
        logging.error(error)

    return date_format

In [40]:

'''Languages include
English, French, German, Dutch, Danish, Hungarian, Norwegian, Polish, Swedish'''

months_name2int = {
    1: ("january", "janvier", "januar", "januari", "januar", "január", "januar", "styczeń", "januari"),
    2: ("february", "février", "februar", "februari", "februar", "február", "februar", "luty", "februari"),
    3: ("march", "mars", "märz", "maart", "marts", "március", "mars", "marzec", "mars"),
    4: ("april", "avril", "april", "april", "april", "április", "april", "kwiecień", "april"),
    5: ("may", "mai", "mai", "mei", "maj", "május", "mai", "maj", "maj"),
    6: ("june", "juin", "juni", "juni", "juni", "június", "juni", "czerwiec", "juni"),
    7: ("july", "juillet", "juli", "juli", "juli", "július", "juli", "lipiec", "juli"),
    8: ("august", "août", "august", "augustus", "august", "augusztus", "august", "sierpień", "augusti"),
    9: ("september", "septembre", "september", "september", "september", "szeptember", "september", "wrzesień",
        "september"),
    10: ("october", "octobre", "oktober", "oktober", "oktober", "október", "oktober", "październik", "oktober"),
    11: ("november", "novembre", "november", "november", "november", "november", "november", "listopad", "november"),
    12: ("december", "décembre", "dezember", "december", "december", "december", "desember", "grudzień", "december")
}


def get_date_format(dates):

    current_year = int(datetime.now().year)

    year_indices = [index for date in dates for index, value in enumerate(date.split('/')) if len(value) == 4
                    and current_year-10 <= int(value) <= current_year+2]
    date_format = '%d/%m/%Y' if any(year_indices) else '%d/%m/%y'

    year_index = max(np.unique(year_indices), key=year_indices.count) if year_indices else 2

    try:
        if year_index == 0:
            for date in dates:
                year, month, day = date.split('/')
                if 0 < int(month) <= 12 and 0 < int(day) <= 31:
                    date_format = '%Y/%m/%d'
                    break

        elif year_index == 2:
            for date in dates:
                day, month, year = date.split('/')
                # print('day: {}, month: {}, year:{}'.format(day, month, year))
                if int(month) > 12 >= int(day) >= 1:
                    date_format = '%m/%d/%Y' if len(year) == 4 else '%m/%d/%y'
                    break

    except Exception as error:
        logging.error(error)

    return date_format


def extract_date(text_segment):
    date = []

    try:
        extracted_month = find_month(text_segment)
        if extracted_month:
            extracted_numbers = re.findall(r'\d+', text_segment)
            day = extracted_numbers[0]
            month = extracted_month[0]
            year = extracted_numbers[1] if 2 <= len(extracted_numbers) <= 4 \
                else datetime.now().year

            if len(day) == 4:
                day, year = year, day
            formatted_date = f'{day}/{month}/{year}'
            date.append(formatted_date)
    except Exception as error:
        logging.error('Extraction failed for %s: %s' % (text_segment, str(error)))

    return date


def find_month(text_segment):
    month_segment = ''.join(re.findall(r'[a-zA-Z]', text_segment)).lower()
    month_int = [key for key, months in months_name2int.items()
                 if month_segment and any(month_segment in month for month in months)]

    return month_int